Podklady pro články Anety Martínkové na iROZHLAS.cz v prosinci 2023.

In [1]:
import os
import pandas as pd

In [2]:
from src.tabulka import tabulka
from src.graf import graf

In [3]:
df = pd.read_csv(os.path.join("data","CZ_RADIO_TOP_100_tydenni.csv"))
wikid = pd.read_json(os.path.join("wikidata","wikidata.json"))
df = df.merge(wikid, on="interpret_hlavni", how="outer")

In [4]:
df = df.sort_values(by=['rok','tyden'])

## Podíl žen na sólovém interpretstvu

In [5]:
roky = [int(x) for x in df['rok'].drop_duplicates().dropna().tolist()]

In [6]:
roky

[2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2020,
 2021,
 2022,
 2023]

In [7]:
pocet_lidi = len(df[(df['velikost'] == 'člověk') & (~df['gender'].isnull())]['interpret_hlavni'].drop_duplicates())

In [8]:
nebin = ['genderfluid','nebinární','genderqueer']

In [9]:
df[(df['velikost'] == 'člověk') & (df['gender'].isin(nebin))][['interpret_hlavni','gender']].drop_duplicates()

,interpret_hlavni,gender
38786,Miley Cyrus,genderfluid
60221,Demi Lovato,genderfluid
62752,Sam Smith,nebinární
91430,Ashley O,genderfluid
91431,Ashnikko,genderfluid
91616,Kim Dracula,nebinární
91618,King Princess,genderqueer
91643,Lil Uzi Vert,nebinární


In [10]:
", ".join(df[(df['velikost'] == 'člověk') & (df['gender'].isin(nebin))][['interpret_hlavni','gender']].drop_duplicates()['interpret_hlavni'].to_list())

'Miley Cyrus, Demi Lovato, Sam Smith, Ashley O, Ashnikko, Kim Dracula, King Princess, Lil Uzi Vert'

In [11]:
podil_zen = pd.Series(df[(df['gender'] == 'žena')].groupby('rok').size() / df[df['gender'].isin(['žena','muž','genderfluid','nebinární','genderqueer'])].groupby('rok').size(), name="podíl žen").reindex(roky)

In [12]:
podil_muzu = pd.Series(df[(df['gender'] == 'muž')].groupby('rok').size() / df[df['gender'].isin(['žena','muž','genderfluid','nebinární','genderqueer'])].groupby('rok').size(), name="podíl mužů").reindex(roky)

In [13]:
podil_ostatnich = pd.Series(df[(df['gender'].isin(['genderfluid','nebinární','genderqueer']))].groupby('rok').size() / df[df['gender'].isin(['žena','muž','genderfluid','nebinární','genderqueer'])].groupby('rok').size(), name="podíl genderfluid a nebinárních osob").reindex(roky)

In [119]:
graf(sloupcovy=[podil_ostatnich,podil_muzu,podil_zen], target="genderovy_podil", titulek='Zastoupení žen v rádiových hitparádách dekádu klesalo', podtitulek=f'''Genderové podíly sólově vystupujícího interpretstva v týdenních hitparádách TOP 100 radio. Z {pocet_lidi} lidí v datech se jinak než jako žena či muž identifikují Miley Cyrus (Ashley O), Demi Lovato, Sam Smith, Ashnikko, Kim Dracula, King Princess a Lil Uzi Vert.''', kredity=["Zdroj dat: ČNS IFPI a Wikidata | Vizualizace: Michal Kašpárek, iROZHLAS.cz","https://www.irozhlas.cz/zpravy-tag/datova-zurnalistika"], procenta=[podil_ostatnich,podil_muzu,podil_zen], prvni=True, osay=' ', osay2=' ', skladany=True, naopak=False)

Graf uložen.


## Věk mužů a žen: čísla

In [15]:
df['vek'] = df['rok'] - df['datum_narozeni']

In [16]:
df[df['gender'] == 'žena']['vek'].median()

27.0

In [17]:
df[df['gender'] == 'žena']['vek'].mean()

29.186850674523182

In [18]:
df[df['gender'] == 'muž']['vek'].median()

31.0

In [19]:
df[df['gender'] == 'muž']['vek'].mean()

33.344492054990184

## Věk mužů a žen: graf

In [20]:
veky = [x for x in range(15,66)]

In [21]:
veky_z = pd.Series(df[df['gender'] == 'žena'].groupby('vek').size(),name='věky žen').reindex(veky)

In [22]:
veky_m = pd.Series(df[df['gender'] == 'muž'].groupby('vek').size(),name='věky mužů').reindex(veky)

In [114]:
graf(sloupcovy=[veky_z,veky_m], target="histogram_stari", titulek='Ženy mizí z rádiové hitparády dřív než muži', podtitulek=f'''Do 24 let mají sólově vystupující ženy stejnou šanci na prosazení jako sólově vystupující muži. Zatímco ženy se pak z rádií vytrácejí, muži se drží o dekádu déle. (Do součtů počítáme každou píseň a každý týden v hitparádě zvlášť. Pro přehlednost ořezáváme sporadicky osídlené okraje grafu.)''', kredity=["Zdroj dat: ČNS IFPI a Wikidata | Vizualizace: Michal Kašpárek, iROZHLAS.cz","https://www.irozhlas.cz/zpravy-tag/datova-zurnalistika"], histogram=True, prvni=True, osay=' ', osay2=' ', skladany=False, naopak=False, barvy=["#fd7f6f","#7eb0d5"])

Graf uložen.


## Úspěšné ženy

In [72]:
top10zen = pd.DataFrame(df[(df['gender'] == 'žena') & (df['pozice'] == 1)].groupby("interpret_hlavni")['week'].nunique().nlargest(10)).reset_index().rename(columns={'interpret_hlavni':'Interpretka','week':'Týdnů jako #1'})

In [73]:
top10zen

,Interpretka,Týdnů jako #1
0,Adele,24
1,Nelly Furtado,24
2,P!Nk,24
3,Shakira,16
4,Ava Max,12
5,Katy Perry,12
6,Alesha Dixon,11
7,Rihanna,10
8,Lady Gaga,8
9,Colbie Caillat,7


In [80]:
top10zen['Země původu'] = top10zen['Interpretka'].apply(lambda x: wikid[wikid['interpret_hlavni'] == x]['zeme_puvodu'].iloc[0])
top10zen['Žánry'] = top10zen['Interpretka'].apply(lambda x: ", ".join([x for x in wikid[wikid['interpret_hlavni'] == x].explode("zanry")['zanry'].drop_duplicates().to_list()[0:3] if x != "pop music"]))
top10zen['Žánry'] = top10zen['Žánry'].apply(lambda x: "-" if len(x) < 3 else x)
top10zen = top10zen.apply(lambda x: x.replace({"Spojené státy americké": "USA"}))

In [81]:
top10zen

,Interpretka,Týdnů jako #1,Země původu,Žánry
0,Adele,24,Spojené království,"bílý soul, rhythm and blues"
1,Nelly Furtado,24,Kanada,"rhythm and blues, lidová hudba"
2,P!Nk,24,USA,"pop rock, contemporary R&B"
3,Shakira,16,Kolumbie,"Latin pop, pop rock"
4,Ava Max,12,USA,dance-pop
5,Katy Perry,12,USA,"rock, pop rock"
6,Alesha Dixon,11,Spojené království,-
7,Rihanna,10,Barbados,"taneční hudba, contemporary R&B"
8,Lady Gaga,8,USA,"elektropop, dance-pop, synthpop"
9,Colbie Caillat,7,USA,"pop rock, country"


In [25]:
df[(df['gender'] == 'žena')].groupby("interpret_hlavni")['track'].nunique().nlargest(10)

interpret_hlavni
Rihanna              30
P!Nk                 28
Lady Gaga            25
Katy Perry           24
Ewa Farna            22
Lucie Vondráčková    18
Avril Lavigne        17
Taylor Swift         17
Britney Spears       16
Lucie Bílá           16
Name: track, dtype: int64

In [26]:
df[(df['gender'] == 'žena') & (df['zeme_puvodu'] == "Česko")].groupby("interpret_hlavni")['track'].nunique().nlargest(10)

interpret_hlavni
Ewa Farna             22
Lucie Vondráčková     18
Lucie Bílá            16
Anna K.               15
Lenny                 15
Aneta Langerová       12
Tereza Kerndlová      11
Olga Lounová           9
Markéta Konvičková     7
Emma Smetana           6
Name: track, dtype: int64

In [27]:
df[(df['gender'] == 'žena') & (df['pozice'] == 1) & (df['zeme_puvodu'] == "Česko")].groupby("interpret_hlavni")['week'].nunique().nlargest(10)

interpret_hlavni
Lenny            4
Ewa Farna        2
Helena Zeťová    1
Name: week, dtype: int64

In [85]:
df[(df['interpret_hlavni'] == 'Miley Cyrus') & (df['pozice'] == 1)]['week'].nunique()

27

In [117]:
tabulka(top10zen, titulek="Ženy, které strávily nejvíce týdnů na vrcholu české hitparády", podtitulek="Pokud bychom tabulku rozšířili o lidi, kteří se neidentifikují jako muži, její čelo by s 27 týdny patřilo Miley Cyrus. (Data od ledna 2006 do listopadu 2023.)", poradi=False)

'<table class="dataframe table table--responsive table--w100p table--striped-red table--plain"><caption>Ženy, které strávily nejvíce týdnů na vrcholu české hitparády</caption><thead><tr style="text-align: right;"><th class="text-nowrap">Interpretka</th><th class="text-nowrap">Týdnů jako #1</th><th class="text-nowrap">Země původu</th><th class="text-nowrap">Žánry</th></tr></thead><tbody class="text-sm"><tr><td>Adele</td><td>24</td><td>Spojené království</td><td>bílý soul, rhythm and blues</td></tr><tr><td>Nelly Furtado</td><td>24</td><td>Kanada</td><td>rhythm and blues, lidová hudba</td></tr><tr><td>P!Nk</td><td>24</td><td>USA</td><td>pop rock, contemporary R&amp;B</td></tr><tr><td>Shakira</td><td>16</td><td>Kolumbie</td><td>Latin pop, pop rock</td></tr><tr><td>Ava Max</td><td>12</td><td>USA</td><td>dance-pop</td></tr><tr><td>Katy Perry</td><td>12</td><td>USA</td><td>rock, pop rock</td></tr><tr><td>Alesha Dixon</td><td>11</td><td>Spojené království</td><td>-</td></tr><tr><td>Rihanna</td

In [109]:
top10cesek = pd.DataFrame(df[(df['gender'] == 'žena') & (df['zeme_puvodu'] == "Česko")].groupby("interpret_hlavni")['week'].nunique().nlargest(10)).reset_index().rename(columns={'interpret_hlavni':'Interpretka','week':'Týdnů v TOP 100'})

In [96]:
df[df['interpret_hlavni'] == 'Ewa Farna'].groupby('track')['pozice'].min().nsmallest(1).index.to_list()[0]

'Vánoce na míru'

In [110]:
top10cesek['Nejvýše umístěný track'] = top10cesek['Interpretka'].apply(lambda x: f"""{df[df['interpret_hlavni'] == x].groupby('track')['pozice'].min().nsmallest(1).index.to_list()[0]} ({int(df[df['interpret_hlavni'] == x]['pozice'].min())}.)""")

In [113]:
tabulka(top10cesek, titulek="Češky, které se v TOP 100 držely nejvíce týdnů", podtitulek="Data od ledna 2006 do listopadu 2023.", poradi=True)

'<table class="dataframe table table--responsive table--w100p table--striped-red table--plain"><caption>Češky, které se v TOP 100 držely nejvíce týdnů</caption><thead><tr style="text-align: right;"><th class="text-nowrap"></th><th class="text-nowrap">Interpretka</th><th class="text-nowrap">Týdnů v TOP 100</th><th class="text-nowrap">Nejvýše umístěný track</th></tr></thead><tbody class="text-sm"><tr><td>1.</td><td>Anna K.</td><td>391</td><td>Údolí včel (6.)</td></tr><tr><td>2.</td><td>Ewa Farna</td><td>304</td><td>Vánoce na míru (1.)</td></tr><tr><td>3.</td><td>Lenny</td><td>285</td><td>Hell.o (1.)</td></tr><tr><td>4.</td><td>Lucie Vondráčková</td><td>230</td><td>Láska umí víc (10.)</td></tr><tr><td>5.</td><td>Lucie Bílá</td><td>225</td><td>Dobrý kafe (7.)</td></tr><tr><td>6.</td><td>Aneta Langerová</td><td>197</td><td>Srdcotepec (24.)</td></tr><tr><td>7.</td><td>Olga Lounová</td><td>196</td><td>Cyklista (14.)</td></tr><tr><td>8.</td><td>Tereza Kerndlová</td><td>139</td><td>Anděl (3.)</

In [112]:
top10cesek

,Interpretka,Týdnů v TOP 100,Nejvýše umístěný track
0,Anna K.,391,Údolí včel (6.)
1,Ewa Farna,304,Vánoce na míru (1.)
2,Lenny,285,Hell.o (1.)
3,Lucie Vondráčková,230,Láska umí víc (10.)
4,Lucie Bílá,225,Dobrý kafe (7.)
5,Aneta Langerová,197,Srdcotepec (24.)
6,Olga Lounová,196,Cyklista (14.)
7,Tereza Kerndlová,139,Anděl (3.)
8,Lenka Filipová,108,Samotní nejsme nic (10.)
9,Helena Zeťová,107,Impossible (Unstoppable) (1.)


## Vs. stream (27. 11. 2023)

In [33]:
stream = pd.read_csv(os.path.join("data","CZ_SINGLES_DIGITAL_TOP_100_tydenni.csv"))

In [34]:
stream = stream.merge(wikid, on="interpret_hlavni", how="outer")

Kolik dat vlastně máme?

In [35]:
len(stream[~stream['velikost'].isnull()]) / len(stream)

0.9520517395182873

In [36]:
len(stream[~stream['gender'].isnull()]) / len(stream)

0.7084786310923689

In [37]:
len(stream[(stream['rok'] == 2023) &  (~stream['gender'].isnull())]) / len(stream[stream['rok'] == 2023])

0.7085106382978723

In [38]:
s_roky = [x for x in roky if x >= 2014]

In [39]:
s_podil_zen = pd.Series(stream[(stream['gender'] == 'žena')].groupby('rok').size() / stream[stream['gender'].isin(['žena','muž','genderfluid','nebinární','genderqueer'])].groupby('rok').size(), name="podíl žen").reindex(s_roky)

In [40]:
s_podil_muzu = pd.Series(stream[(stream['gender'] == 'muž')].groupby('rok').size() / stream[stream['gender'].isin(['žena','muž','genderfluid','nebinární','genderqueer'])].groupby('rok').size(), name="podíl mužů").reindex(s_roky)

In [41]:
s_podil_ostatnich = pd.Series(stream[(stream['gender'].isin(['genderfluid','nebinární','genderqueer']))].groupby('rok').size() / stream[stream['gender'].isin(['žena','muž','genderfluid','nebinární','genderqueer'])].groupby('rok').size(), name="podíl genderfluid a nebinárních osob").reindex(s_roky)

In [120]:
graf(sloupcovy=[s_podil_ostatnich,s_podil_muzu,s_podil_zen], target="genderovy_podil_stream", titulek='Ženy prohrávají s muži i na streamu', podtitulek=f'''Genderové podíly sólově vystupujícího interpretstva v žebříčku TOP 100 digitál.''', kredity=["Zdroj dat: ČNS IFPI a Wikidata | Vizualizace: Michal Kašpárek, iROZHLAS.cz","https://www.irozhlas.cz/zpravy-tag/datova-zurnalistika"], procenta=[s_podil_ostatnich,s_podil_muzu,s_podil_zen], prvni=True, osay=' ', osay2=' ', skladany=True, naopak=False)

Graf uložen.
